Adaboost Decision Stumps

In [6]:
import os
import scipy.io as spio
import numpy as np
from os.path import join, isdir, isfile
%matplotlib inline
import matplotlib.pyplot as plt

DATA_DIR = '../data'
# 'file_name' 'class_name' 'class' 'eng' 'mfc' 'chroma' 't'
# 'keystrength' 'brightness' 'zerocross' 'roughness' 'inharmonic' 'hcdf'
#

FVs = join(DATA_DIR, 'FV.mat')
LBs = join(DATA_DIR, 'LB.mat')
if not isfile(FVs) or not isfile(LBs):
    print('Generating Fisher Vectors')
    !matlab -nodisplay -nosplash -nodesktop -r "run('../tools/FV_concat.m');exit;"
else:
    print('Using existing FVs')

mfcc = np.transpose(spio.loadmat(FVs)['FV'])
labels = spio.loadmat(LBs)['LB'][0]

height = mfcc.shape[0]
width = mfcc.shape[1]
w = int(width/2)
kernelized = np.zeros((height, w))

for i in range(height):
    for j in range(w):
        kernelized[i, j] = mfcc[i, j]
        #kernelized[i, width+j] = mfcc[i,j]*mfcc[i, j]
mfcc = kernelized

N = mfcc.shape[0]

p = np.random.permutation(N)
mfcc = mfcc[p]
labels = labels[p]

N

Using existing FVs


1000

In [7]:
trN = int(0.8 * N)
teN = N - trN
trX, trY = mfcc[:trN], labels[:trN]
teX, teY = mfcc[-teN:], labels[-teN:]
len(set(trY))

means = np.mean(trX, axis=0)
stddevs = np.std(trX, axis=0)

def summary(x):
    return '[{:.4f}, {:.4f}]'.format(np.ndarray.min(x), np.ndarray.max(x))
print('means in range', summary(means))
print('std in range', summary(stddevs))

# Whitened data
def adjust(x): return (x - means[None, :]) / stddevs[None, :]
trXadj = adjust(trX)
teXadj = adjust(teX)

print('norm. means in range', summary(np.mean(trXadj, axis=0)))
print('norm. std in range', summary(np.std(trXadj, axis=0)))
trX.shape

means in range [-0.0193, 0.0077]
std in range [0.0244, 0.0613]
norm. means in range [-0.0000, 0.0000]
norm. std in range [1.0000, 1.0000]


(800, 480)

In [10]:
from sklearn.cross_validation import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.tree import DecisionTreeClassifier

for n in range(1, 11):
    numStumps = 2*n
    stumps = AdaBoostClassifier(n_estimators=numStumps, base_estimator=DecisionTreeClassifier(max_depth=1))
    stumps.fit(trX, trY)
    print('n_classes {}'.format(stumps.n_classes_))
    print('numStumps {}, train data accuracy {}, test accuracy {}'.format(numStumps, stumps.score(trX, trY), stumps.score(teX, teY)))

n_classes 10
numStumps 2, train data accuracy 0.2475, test accuracy 0.24
n_classes 10
numStumps 4, train data accuracy 0.2975, test accuracy 0.25
n_classes 10
numStumps 6, train data accuracy 0.28875, test accuracy 0.255
n_classes 10
numStumps 8, train data accuracy 0.30125, test accuracy 0.255
n_classes 10
numStumps 10, train data accuracy 0.295, test accuracy 0.25
n_classes 10
numStumps 12, train data accuracy 0.29125, test accuracy 0.25
n_classes 10
numStumps 14, train data accuracy 0.30625, test accuracy 0.255
n_classes 10
numStumps 16, train data accuracy 0.30375, test accuracy 0.255
n_classes 10
numStumps 18, train data accuracy 0.28, test accuracy 0.27
n_classes 10
numStumps 20, train data accuracy 0.28, test accuracy 0.27


In [ ]:
reg.fit(trXadj, trY)
print('train data accuracy {} test accuracy {}'.format(reg.score(trXadj, trY), reg.score(teXadj, teY)))

In [ ]:
reg.fit(pcaX, trY)
print('train data accuracy {} test accuracy {}'.format(reg.score(pcaX, trY), reg.score(pca.transform(teX), teY)))
# interesting, training went up... try different PCA var exp values.

In [ ]:
# Random-restart softmax
from sklearn.linear_model import LogisticRegressionCV
import warnings
warnings.filterwarnings('ignore')

reg = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs', max_iter=5000, n_jobs=8)
reg.fit(trX, trY)
print('train data accuracy {} test accuracy {}'.format(reg.score(trX, trY), reg.score(teX, teY)))